In [2]:
! pip -qqq install yt-dlp

In [3]:
from yt_dlp import YoutubeDL
import os 
import re 

In [4]:
class YouTubeAudioDownloader:
    def __init__(self, output_folder):
        self.output_folder = os.path.abspath(output_folder)
        self.audio_files_dict = {}

    def get_safe_filename(self, filename):
        safe_filename = re.sub(r'[^\w\-.]', '_', filename)
        safe_filename = re.sub(r'_+', '_', safe_filename)
        safe_filename = safe_filename[:50].strip('_')
        return safe_filename

    def download_audio(self, video_url):
        try:
            ydl_opts = {
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'mp3',
                    'preferredquality': '192',
                }],
                'outtmpl': os.path.join(self.output_folder, '%(title)s.%(ext)s'),
            }

            with YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(video_url, download=True)
                filename = ydl.prepare_filename(info)
                base, ext = os.path.splitext(filename)
                new_file = base + '.mp3'

            print(f"Audio file downloaded: {new_file}")
            self.audio_files_dict[video_url] = new_file
            return new_file
        except Exception as e:
            print(f"Error downloading audio from {video_url}: {str(e)}")
            return None

    def download_multiple_audios(self, video_urls):
        for url in video_urls:
            print(f"Processing video: {url}")
            audio_file = self.download_audio(url)
            if audio_file is None:
                print(f"Failed to download audio from video: {url}")
        return self.audio_files_dict

In [5]:
downloader = YouTubeAudioDownloader(output_folder="C://GEN AI//myenv1//MULTIMODAL_RAG//audio")

# List of video URLs
video_urls = ["https://www.youtube.com/watch?v=qP1JKWBBy80",
                "https://www.youtube.com/watch?v=_p58cZIHDG4"]

# Download audios from multiple videos
audio_files = downloader.download_multiple_audios(video_urls)

print("Downloaded audio files:")
for audio_file in audio_files:
    print(audio_file)


Processing video: https://www.youtube.com/watch?v=qP1JKWBBy80
[youtube] Extracting URL: https://www.youtube.com/watch?v=qP1JKWBBy80
[youtube] qP1JKWBBy80: Downloading webpage
[youtube] qP1JKWBBy80: Downloading ios player API JSON
[youtube] qP1JKWBBy80: Downloading mweb player API JSON
[youtube] qP1JKWBBy80: Downloading m3u8 information
[info] qP1JKWBBy80: Downloading 1 format(s): 251
[download] C:\GEN AI\myenv1\MULTIMODAL_RAG\audio\How to Invest with your Conscience： ESG Investing.webm has already been downloaded
[download] 100% of    4.32MiB
[ExtractAudio] Destination: C:\GEN AI\myenv1\MULTIMODAL_RAG\audio\How to Invest with your Conscience： ESG Investing.mp3
Deleting original file C:\GEN AI\myenv1\MULTIMODAL_RAG\audio\How to Invest with your Conscience： ESG Investing.webm (pass -k to keep)
Audio file downloaded: C:\GEN AI\myenv1\MULTIMODAL_RAG\audio\How to Invest with your Conscience： ESG Investing.mp3
Processing video: https://www.youtube.com/watch?v=_p58cZIHDG4
[youtube] Extracting

In [6]:
audio_files

{'https://www.youtube.com/watch?v=qP1JKWBBy80': 'C:\\GEN AI\\myenv1\\MULTIMODAL_RAG\\audio\\How to Invest with your Conscience： ESG Investing.mp3',
 'https://www.youtube.com/watch?v=_p58cZIHDG4': "C:\\GEN AI\\myenv1\\MULTIMODAL_RAG\\audio\\ESG investing is 'a complete fraud'： Venture capitalist Chamath Palihapitiya.mp3"}

In [8]:
# transcription generations with whisper 


In [13]:
%pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to c:\users\hp\appdata\local\temp\pip-req-build-cz75k9ej
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numba-0.60.0-cp312-cp312-win_amd64.whl.metadata (2.8 kB)
  Using cached torch-2.4.1-cp312-cp312-win_amd64.whl.metadata (27 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached more_itertools-10.5.0-py3-none-any.whl.metadata (36 kB)
  Using cached tiktoken-0.8.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached llvmlite-0.43.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached numpy-2.0.2-cp312-cp312-win

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\hp\AppData\Local\Temp\pip-req-build-cz75k9ej'


In [15]:
import whisper 
import torch 

In [16]:
class AudioTranscriber:
    def __init__(self, input_folder):
        self.input_folder = os.path.abspath(os.path.join(os.getcwd(), input_folder))
        self.whisper_model = None
        self.transcriptions_dict = {}

    def transcribe_audio(self, audio_file):
        try:
            if not os.path.exists(audio_file):
                print(f"Audio file not found: {audio_file}")
                return None

            file_size = os.path.getsize(audio_file)
            if file_size == 0:
                print(f"Audio file is empty: {audio_file}")
                return None

            transcription = self.whisper_model.transcribe(audio_file)
            return transcription["text"]
        except Exception as e:
            print(f"Error in transcribe_audio: {str(e)}")
            return None

    def transcribe_all_audios(self, audio_files_dict):
        for url, audio_path in audio_files_dict.items():
            if not audio_path.endswith('.mp3'):
                print(f"Skipping non-mp3 file: {audio_path}")
                continue

            transcription = self.transcribe_audio(audio_path)

            if transcription is not None:
                
                self.transcriptions_dict[url] = {
                    'url': url,
                    'audio_path': audio_path,
                    'transcription': transcription
                }
            else:
                print(f"Failed to transcribe audio: {audio_path}")

        return self.transcriptions_dict

In [17]:

device = "cuda" if torch.cuda.is_available() else "cpu"


whisper_model = whisper.load_model("small", device=device)


100%|███████████████████████████████████████| 461M/461M [03:08<00:00, 2.56MiB/s]
d:\GEN AI\myenv1\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experiment

In [18]:
# Initialize the AudioTranscriber
transcriber = AudioTranscriber(input_folder="C://GEN AI//myenv1//MULTIMODAL_RAG//audio")

# Initialize your Whisper model
transcriber.whisper_model = whisper_model

# Transcribe all audios in the input folder
transcriptions_dict = transcriber.transcribe_all_audios(audio_files)

for url, data in transcriptions_dict.items():
    print(f"URL: {url}")
    print(f"Audio file: {data['audio_path']}")
    print(f"Transcription: {data['transcription'][:100]}...") 
    print("---")

d:\GEN AI\myenv1\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


URL: https://www.youtube.com/watch?v=qP1JKWBBy80
Audio file: C:\GEN AI\myenv1\MULTIMODAL_RAG\audio\How to Invest with your Conscience： ESG Investing.mp3
Transcription:  It's your money, why not invest it in something that's truly rooted in your values? Hey there, welc...
---
URL: https://www.youtube.com/watch?v=_p58cZIHDG4
Audio file: C:\GEN AI\myenv1\MULTIMODAL_RAG\audio\ESG investing is 'a complete fraud'： Venture capitalist Chamath Palihapitiya.mp3
Transcription:  ESG, real or marketing? It's a complete fraud. Complete fraud. It's so ridiculous. Governance has b...
---


In [25]:
transcriptions_dict

{'https://www.youtube.com/watch?v=qP1JKWBBy80': {'url': 'https://www.youtube.com/watch?v=qP1JKWBBy80',
  'audio_path': 'C:\\GEN AI\\myenv1\\MULTIMODAL_RAG\\audio\\How to Invest with your Conscience： ESG Investing.mp3',
  'transcription': " It's your money, why not invest it in something that's truly rooted in your values? Hey there, welcome back to my channel. It's another day of interviewing John Mullet, my colleague here at Pleasant Wealth. Thanks for being here again, John. Thanks Liz, my pleasure. It's great to be able to have some of these conversations about what we talk about here in the office here for you so you can make some good decisions with your investments. So in today's video we're going to be talking about ESG investing and you may not have heard that term but stay tuned because that's what we're going to talk about. Before we get into that, don't forget to subscribe to this channel for regular Pleasant Financial Converses. Alright John, let's talk about ESG investing.

In [40]:
first_item = next(iter(transcriptions_dict.items()))
first_transcription = first_item[1]['transcription']
print(first_transcription)


 It's your money, why not invest it in something that's truly rooted in your values? Hey there, welcome back to my channel. It's another day of interviewing John Mullet, my colleague here at Pleasant Wealth. Thanks for being here again, John. Thanks Liz, my pleasure. It's great to be able to have some of these conversations about what we talk about here in the office here for you so you can make some good decisions with your investments. So in today's video we're going to be talking about ESG investing and you may not have heard that term but stay tuned because that's what we're going to talk about. Before we get into that, don't forget to subscribe to this channel for regular Pleasant Financial Converses. Alright John, let's talk about ESG investing. What does ESG even stand for? So ESG stands for Environmental Social Governance. Environmental is a pretty self-explanatory, anything to do with being a good steward of the environment, natural resources, sustainability. And social has to

In [38]:
import textwrap
wrapped_transcription = textwrap.fill(first_transcription, width=80)
print(wrapped_transcription)


 It's your money, why not invest it in something that's truly rooted in your
values? Hey there, welcome back to my channel. It's another day of interviewing
John Mullet, my colleague here at Pleasant Wealth. Thanks for being here again,
John. Thanks Liz, my pleasure. It's great to be able to have some of these
conversations about what we talk about here in the office here for you so you
can make some good decisions with your investments. So in today's video we're
going to be talking about ESG investing and you may not have heard that term but
stay tuned because that's what we're going to talk about. Before we get into
that, don't forget to subscribe to this channel for regular Pleasant Financial
Converses. Alright John, let's talk about ESG investing. What does ESG even
stand for? So ESG stands for Environmental Social Governance. Environmental is a
pretty self-explanatory, anything to do with being a good steward of the
environment, natural resources, sustainability. And social has to

In [27]:
import json

audio_data = [
    {
        "url": value["url"],
        "audio_path": value["audio_path"],
        "transcription": value["transcription"]
    }
    for value in transcriptions_dict.values()
]


print(json.dumps(audio_data, indent=2))

[
  {
    "url": "https://www.youtube.com/watch?v=qP1JKWBBy80",
    "audio_path": "C:\\GEN AI\\myenv1\\MULTIMODAL_RAG\\audio\\How to Invest with your Conscience\uff1a ESG Investing.mp3",
    "transcription": " It's your money, why not invest it in something that's truly rooted in your values? Hey there, welcome back to my channel. It's another day of interviewing John Mullet, my colleague here at Pleasant Wealth. Thanks for being here again, John. Thanks Liz, my pleasure. It's great to be able to have some of these conversations about what we talk about here in the office here for you so you can make some good decisions with your investments. So in today's video we're going to be talking about ESG investing and you may not have heard that term but stay tuned because that's what we're going to talk about. Before we get into that, don't forget to subscribe to this channel for regular Pleasant Financial Converses. Alright John, let's talk about ESG investing. What does ESG even stand for?